In [1]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM
from modeling_git import GitForCausalLMClipEmb
processor = AutoProcessor.from_pretrained("/userhome2/liweile/EEG_Image_decode/Generation/git-large-coco/")
clip_text_model = GitForCausalLMClipEmb.from_pretrained("/userhome2/liweile/EEG_Image_decode/Generation/git-large-coco/")
clip_text_model.to(device) # if you get OOM running this script, you can switch this to cpu and lower minibatch_size to 4
clip_text_model.eval().requires_grad_(False)

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# git_image = Image.open("/root/autodl-tmp/Workspace/EEG_caption/docs/test/banana_09s.jpg")
# pixel_values = processor(images=git_image, return_tensors="pt").pixel_values.to(device)
# vision_encoder=model.git.image_encoder

# git_image_features=vision_encoder(pixel_values).last_hidden_state.cpu()
# git_image_features.shape


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
GitForCausalLMClipEmb has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `

GitForCausalLMClipEmb(
  (git): GitModelClipEmb(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(257, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-23): 24 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bia

In [3]:
git_test = torch.load("/userhome2/liweile/EEG_Image_decode/Generation/variables/ViT-L-14_features_multimodal_test.pt",weights_only=True)['img_features']
git_test.shape

torch.Size([200, 256, 1024])

In [4]:
emb_eeg_test = torch.load('/userhome2/liweile/EEG_Image_decode/Reconstruction/ATM_S_eeg_features_sub-01_test.pt', weights_only=True).unsqueeze(1)
emb_eeg_test.shape

torch.Size([200, 1, 1024])

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import open_clip
from matplotlib.font_manager import FontProperties

import sys
from diffusion_prior import *
from custom_pipeline import *
# os.environ["CUDA_VISIBLE_DEVICES"] = "5" 
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from einops.layers.torch import Rearrange, Reduce

# Define the neural network
class PixelProjector(nn.Sequential):
    def __init__(self, proj_dim=1024):
        super().__init__(
            Rearrange('B C L->B L C'),    
            nn.Linear(1, 256),
            nn.LayerNorm(256),
            Rearrange('B L C->B C L'),
            nn.Linear(1024, 1024),
            nn.LayerNorm(proj_dim),
            )
model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
model.load_state_dict(torch.load('/userhome2/liweile/EEG_Image_decode/Generation/EEG_caption/model_weights/PixelProjector_best.bin'))


/tmp/ipykernel_481689/2052554021.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/userhome2/liweile/EEG_Image_decode/Generation/EEG_ca

<All keys matched successfully>

In [6]:
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)
# number of parameters
print(sum(p.numel() for p in diffusion_prior.parameters() if p.requires_grad))
pipe = Pipe(diffusion_prior, device=device)

# load pretrained model
model_name = 'diffusion_prior' # 'diffusion_prior_vice_pre_imagenet' or 'diffusion_prior_vice_pre'
# pipe.train(dl, num_epochs=150, learning_rate=1e-3) # to 0.142

# diffusion_prior.pt 是先运行README中的clip pipeline两个文件才生成的。
pipe.diffusion_prior.load_state_dict(torch.load(r'/userhome2/liweile/EEG_Image_decode/Generation/fintune_ckpts/ATMS/sub-01/diffusion_prior.pt', weights_only=True, map_location=device))

9675648


<All keys matched successfully>

In [21]:
# Loop through each element in emb_eeg_test and generate captions
with open('semantic_level_caption.txt', 'w') as f:
    for emb in emb_eeg_test:
        # Generate h for each emb
        h = pipe.generate(c_embeds=emb, num_inference_steps=4, guidance_scale=5.0)

        # Get test image
        test_img_257_1024 = model(h.unsqueeze(0).to(torch.bfloat16).to(device))
        print(f"test_img_257_1024 shape: {test_img_257_1024.shape}")

        # Generate captions
        generated_ids = clip_text_model.generate(pixel_values=test_img_257_1024.to(device).float(), max_new_tokens=25)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
        print(generated_caption)
        # Write each caption to a new line in the txt file
        f.write(f"{generated_caption[0]}\n")

print("Captions generated and saved to 'generated_captions.txt'.")


4it [00:00, 101.16it/s]


test_img_257_1024 shape: torch.Size([1, 256, 1024])


RuntimeError: output with shape [1, 1, 1, 1] doesn't match the broadcast shape [1, 1, 1, 2]

In [10]:
# dataset = EmbeddingDataset(
#     c_embeddings=eeg_features_train, h_embeddings=emb_img_train_4, 
#     # h_embeds_uncond=h_embeds_imgnet
# )
# dl = DataLoader(dataset, batch_size=1024, shuffle=True, num_workers=64)
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)
# number of parameters
print(sum(p.numel() for p in diffusion_prior.parameters() if p.requires_grad))
pipe = Pipe(diffusion_prior, device=device)

# load pretrained model
model_name = 'diffusion_prior' # 'diffusion_prior_vice_pre_imagenet' or 'diffusion_prior_vice_pre'
# pipe.train(dl, num_epochs=150, learning_rate=1e-3) # to 0.142 
pipe.diffusion_prior.load_state_dict(torch.load(r'/userhome2/liweile/EEG_Image_decode/Generation/fintune_ckpts/ATMS/sub-01/diffusion_prior.pt', weights_only=True, map_location=device))

h = pipe.generate(c_embeds=emb_eeg_test[0], num_inference_steps=4, guidance_scale=5.0)


model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
model.load_state_dict(torch.load('/userhome2/liweile/EEG_Image_decode/Generation/EEG_caption/model_weights/PixelProjector_best.bin'))


test_img_257_1024 = model(h.unsqueeze(0).to(torch.bfloat16).to(device))

generated_ids = clip_text_model.generate(pixel_values=test_img_257_1024.to(device).float(), max_new_tokens=25)

generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
generated_caption

9675648


4it [00:00, 112.28it/s]
/tmp/ipykernel_471896/1309825831.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/userhome2/liweile/EEG_Image_

RuntimeError: output with shape [1, 1, 1, 1] doesn't match the broadcast shape [1, 1, 1, 2]

In [ ]:
# generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# generated_caption